In [6]:
import sys
sys.path.append("../../../../ns3_sem")
import sem
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
ns_path = '../../../../../ns3-dev-hofailure_RACH'
script = 'lena_hofailure'
campaign_dir = 'rngrun1'
campaign = sem.CampaignManager.new(ns_path, script, campaign_dir, overwrite=True)
print(campaign)

--- Campaign info ---
script: lena_hofailure
params: {'simTime': '50', 'MacroPower': 43.0, 'SmallPower': 20.0, 'nBlocks': '1', 'nMacroEnbSites': '1', 'macroUeDensity': '1e-05', 'homeEnbDeploymentRatio': '0.2', 'homeEnbActivationRatio': '0.5', 'homeUesHomeEnbRatio': '1', 'macroEnbDlEarfcn': '100', 'homeEnbDlEarfcn': '100', 'macroEnbBandwidth': '25', 'homeEnbBandwidth': '25', 'generateRem': 'false', 'epc': 'true', 'useUdp': 'true', 'outdoorUeSpeed': 5.0, 'handoverAlgorithm': 'A3', 'Hysteresis': 3.0, 'TimeToTrigger': 256.0, 'ServingCellThreshold': 20.0, 'NeighbourCellOffset': 5.0, 'SrsPeriodicity': 80.0, 'epcDl': 'true', 'epcUl': 'true', 'fadingTrace': '', 'homeEnbTxPowerDbm': '20', 'interSiteDistance': '500', 'macroEnbTxPowerDbm': '43', 'nMacroEnbSitesX': '1', 'numBearersPerUe': '1', 'outdoorUeMaxSpeed': '0', 'outdoorUeMinSpeed': '0', 'remRbId': '-1', 'srsPeriodicity': '80'}
HEAD: 736d80875a8a14c7d82ec804b2c90c43bcdfd718
Runner type: <class 'sem.parallelrunner.ParallelRunner'>
----------

In [7]:
import numpy as np
param_combination = {'simTime':500, 
                     'nBlocks': 4, #list([1,2,3,4,5,6,7,8,9,10]),#round(4*nBlocks*0.1)
                     'nMacroEnbSites': 1, #list([1,2,3,4,5,6,7,8,9,10]),
                     'nMacroEnbSitesX': 1,
                     'homeEnbDeploymentRatio': 1.0,
                     'homeEnbActivationRatio': 1.0,
                     'homeUesHomeEnbRatio': 1.0,
                     'outdoorUeSpeed': 5.0,
                     'handoverAlgorithm': 'A2A4',
                     'ServingCellThreshold':[j for j in np.arange(0.0, 20.0, 1.0)],
                     'NeighbourCellOffset':[i for i in np.arange(0.0, 20.0, 1.0)],
                     'SrsPeriodicity': 80,
                     'generateRem': 0,
                     'RngRun': [1]
                    }

In [8]:
campaign.run_missing_simulations(param_combination, stop_on_errors=False)

Running simulations: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [8:02:59<00:00, 72.45s/simulation]


In [20]:
def get_average_res(result):
    if result['output']['stderr'] != "":
        num_HOs = -1
        num_RLFs = -1
        num_HFJs = -1
        num_HFLs = -1
    else:
        split_results = result['output']['stdout'].split("\n")
        num_HOs = 0
        num_SMhos = 0
        num_SMho_fail = 0
        num_MSho_fail = 0
        num_MShos = 0
        num_RLFs = 0
        num_HFJs = 0
        num_HFLs = 0
    
    for line in split_results:
        num_HOs += line.count("Handover_start")
        num_SMhos += line.count("Small to Macro")
        num_SMho_fail += line.count("targetCellId 1")
        num_MShos += line.count("Macro to Small")
        num_HFJs += line.count("Handover_Failure_JoiningTimeout")
        num_RLFs += line.count("Radio_link_failure_detected")
        for
    #num_MSho_fail = num_HFJs - num_SMho_fail + num_RLFs
    #num_MShos = num_MShos + num_RLFs
    
    return [num_HOs, num_SMhos, num_SMho_fail, 
            num_MShos, num_MSho_fail, num_RLFs, num_HFJs, num_HFLs]

In [21]:
results_rngrun1 = campaign.get_results_as_dataframe(get_simple_result, 
                                            ['num_HOs','num_HFJs'], 
                                            params = param_combination, drop_constant_columns=True)
print(results_rngrun1)

    ServingCellThreshold  NeighbourCellOffset
0                    0.0                  0.0
1                    0.0                  2.0
2                    0.0                  4.0
3                    0.0                  6.0
4                    0.0                  8.0
..                   ...                  ...
95                  18.0                 10.0
96                  18.0                 12.0
97                  18.0                 14.0
98                  18.0                 16.0
99                  18.0                 18.0

[100 rows x 2 columns]


In [ ]:
results_Hys1 = results_rngrun_ave[(results_rngrun_ave['TimeToTrigger'] == 256.0)]
plt.rcParams.update({'font.size':14})
fig = plt.figure(figsize=(7, 5))
ax = plt.axes()
ax.plot(results_Hys1['Hysteresis'], results_Hys1['num_HOs'], color = 'blue')
ax.plot(results_Hys1['Hysteresis'], results_Hys1['num_HFJs'], color = 'red')
#ax.plot(results_Hys['Hysteresis'], results_Hys['num_RLFs'], color = 'black')
ax.grid()
ax.set_xlabel('Hysteresis')
ax.set_ylabel('Number of HOs and HFJs')
ax.set_title('TimeToTrigger = 256 ms, speed = 1 m/s')
ax.legend(['num_HOs', 'num_HFJs','num_RLFs'])

In [28]:
j = [j for j in np.arange(0.0, 5.0, 0.5)]
j

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2000000000000002,
 1.3,
 1.4000000000000001,
 1.5,
 1.6,
 1.7000000000000002,
 1.8,
 1.9000000000000001,
 2.0,
 2.1,
 2.2,
 2.3000000000000003,
 2.4000000000000004,
 2.5,
 2.6,
 2.7,
 2.8000000000000003,
 2.9000000000000004]

In [2]:
import numpy as np
[j for j in np.arange(0.0, 5.0, 0.5)]

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]

In [3]:
[j for j in np.arange(0.0, 10.0, 1.0)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]